# Логистическая регрессия

Импортирую всякие модули

In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split